In [16]:
#Importing dependencies
import pandas as pd
import os

In [17]:
## getDataFrameFromFiles - use pandas to read the csv file and get a dataframe to work with it
# @params: path - the path of the data file
# @return: dataframe created from the file
##
def getDataFrameFromFile(path):
    #Use index_col so pandas doesn't use the fisrt column as an index column.
    return pd.read_csv(path, index_col=False)

In [18]:
## sortDataFrame - Sorts dataframe by column and ascending or descending order
# @params: dataframe - dataframe that will be sorted
#          column - name of column
#          ascending - True for ascending sort. False for descending sort.
# @return: sortedDF - the dataframe sorted
##
def sortDataFrame(dataframe, column, ascending):
    sortedDF = dataframe.sort_values(by=[column], ascending=ascending)

    #Reset the index of the dataframe
    sortedDF.index = [x for x in range(0, len(sortedDF))]
    return sortedDF

In [19]:
## insertEmptyRow - Inserts an empty row in a dataframe on the index defined.
# @params: dataframe - the dataframe in which the empty row will be inserted.
#          index - index where the row will be inserted. Index starts with number for first element of the dataframe.
# @return: df - dataframe with the new empty row inserted.
##
def insertEmptyRow(dataframe, index):
    row = pd.DataFrame(emptyDict, index=[index])
    df = pd.concat([dataframe.iloc[:index - 1], row, dataframe.iloc[index - 1:]], ignore_index=True)
    return df

In [20]:
## compareRowData - checks if the data of two rows is the same or not
# @params: clientRow - first pandas.Series to be compared
#          tutukaRow - pandas.Series which will be compared against.
# @return: Boolean
#
def compareRowData(clientRow, tutukaRow):
    flag = True
    for x in range(0,len(clientRow)):
        if(not (clientRow[x] == tutukaRow[x])):
            flag = False
            break
    return flag

In [21]:
def getMaxSize():
    #Get the length of both files so we can choose the largest file and iterate over that one
    clientLen = len(sortedClient)
    tutukaLen = len(sortedTutuka)
    print(clientLen)
    print(tutukaLen)

    #Assign the biggest length
    dfLen = clientLen if (clientLen > tutukaLen) else tutukaLen
    print(f'Max:{dfLen}\n')
    return dfLen

In [22]:
#csv files path
clientFilePath = os.path.join("..", "Resources", "ClientMarkoffFile20140113.csv")
tutukaFilePath = os.path.join("..", "Resources", "TutukaMarkoffFile20140113.csv")

In [23]:
#Create dataframes for each file
client_df = getDataFrameFromFile(clientFilePath)
tutuka_df = getDataFrameFromFile(tutukaFilePath)

In [24]:
#emptyDict stores in a dictionary all the column names of a dataframe as keys and empty values.
#This variable helps out in the process of creating an empty row using the insertEmptyRow declared before.
emptyDict = {}
for column in client_df.columns:
    emptyDict[column] = ""

In [25]:
#Sort client dataframe by TransactionID (ascending order) so its easier to find possible matches.
sortedClient = sortDataFrame(client_df, "TransactionID", True)

In [26]:
#Sort tutuka dataframe by TransactionID (ascending order)
sortedTutuka = sortDataFrame(tutuka_df, "TransactionID", True)

In [27]:
#We look through each dataframe and search if there are transactions that are the same.
#If we find the same transaction in the same dataframe. We drop it.
#Because is a transaction that is repeated and will cause problems. i.e. calculating the correct total amount the client
#or a reatil store really  sold.

#Drop duplicates in sortedClient dataframe
sortedClient.drop_duplicates(inplace=True)
#Reset the index of the dataframe
sortedClient.index = [x for x in range(0, len(sortedClient))]
sortedClient

,ProfileName,TransactionDate,TransactionAmount,TransactionNarrative,TransactionDescription,TransactionID,TransactionType,WalletReference
0,Card Campaign,2014-01-12 08:24:56,-10250,Sunset Butchery 100343 Francistown BW,DEDUCT,4012304765521,0,P_NzI4Njk3MTBfMTM4MTQ5MTEwNy4wNTMy
1,Card Campaign,2014-01-12 08:41:20,-3475,Choppies Superst102145 Lobatse BW,DEDUCT,4012314602553,0,P_NzU3OTY0MzBfMTM4NzIwMzAxOS4zMzYz
2,Card Campaign,2014-01-12 12:56:25,4310,766831 SHOPRITE GABORONE BOTSWANA BW,REVERSAL,4012321856252,0,P_NzI0MTE0MjJfMTM4ODEzMTA0Mi42MTI3
3,Card Campaign,2014-01-12 12:56:25,-4310,766831 SHOPRITE GABORONE BOTSWANA BW,DEDUCT,4012321856252,0,P_NzI0MTE0MjJfMTM4ODEzMTA0Mi42MTI3
4,Card Campaign,2014-01-12 12:57:31,-4310,766831 SHOPRITE GABORONE BOTSWANA BW,DEDUCT,4012322510401,0,P_NzI0MTE0MjJfMTM4ODEzMTA0Mi42MTI3
...,...,...,...,...,...,...,...,...
299,Card Campaign,2014-01-12 21:59:29,-10000,KFC ATM JOHANNESBURG BW,DEDUCT,584012647694946,1,P_NzI3NDUxNDRfMTM4NDM1MDQ5NC4yMTUx
300,Card Campaign,2014-01-12 22:05:11,-10000,CHOPPIES GABANE ROAD BOT BW,DEDUCT,584012651116258,1,P_NzQzNTQzODFfMTM4NzM2Njg5NS4xODAy
301,Card Campaign,2014-01-12 22:45:14,-10000,HEMAMO GABORONE BW,DEDUCT,584012676142890,1,P_NzY1NTAxNDZfMTM4ODM5MjU4My4xMTQ3
302,Card Campaign,2014-01-12 14:16:24,-10000,TRUWORTHS BOTSWANA BW,DEDUCT,4049540590428544,1,P_NzI3NDUxNDRfMTM4NDM1MDQ5NC4yMTUx


In [28]:
#Drop duplicates in sortedTutuka dataframe
sortedTutuka.drop_duplicates(inplace=True)
#Reset the index of the dataframe
sortedTutuka.index = [x for x in range(0, len(sortedTutuka))]
sortedTutuka

,ProfileName,TransactionDate,TransactionAmount,TransactionNarrative,TransactionDescription,TransactionID,TransactionType,WalletReference
0,Card Campaign,2014-01-12 08:24:56,-10250,Sunset Butchery 100343 Francistown BW,DEDUCT,4012304765521,0,P_NzI4Njk3MTBfMTM4MTQ5MTEwNy4wNTMy
1,Card Campaign,2014-01-12 08:41:20,-3475,Choppies Superst102145 Lobatse BW,DEDUCT,4012314602553,0,P_NzU3OTY0MzBfMTM4NzIwMzAxOS4zMzYz
2,Card Campaign,2014-01-12 12:56:25,4310,766831 SHOPRITE GABORONE BOTSWANA BW,REVERSAL,4012321856252,0,P_NzI0MTE0MjJfMTM4ODEzMTA0Mi42MTI3
3,Card Campaign,2014-01-12 12:56:25,-4310,766831 SHOPRITE GABORONE BOTSWANA BW,DEDUCT,4012321856252,0,P_NzI0MTE0MjJfMTM4ODEzMTA0Mi42MTI3
4,Card Campaign,2014-01-12 12:57:31,-4310,766831 SHOPRITE GABORONE BOTSWANA BW,DEDUCT,4012322510401,0,P_NzI0MTE0MjJfMTM4ODEzMTA0Mi42MTI3
...,...,...,...,...,...,...,...,...
299,Card Campaign,2014-01-12 17:00:28,-5000,*MOLEPOLOLE SPAR MOLEPOLOLE BW,DEDUCT,584012612282599,1,P_NzI3MzE2NTZfMTM4NTY0ODQxMC45Mjk4
300,Card Campaign,2014-01-12 21:55:27,-20000,ENGEN TSOLAMOSESI GABORONE BW,DEDUCT,584012645272003,1,P_NzUwNDg3ODdfMTM4MjUyMDA3Mi44MjM4
301,Card Campaign,2014-01-12 21:59:29,-10000,KFC ATM JOHANNESBURG BW,DEDUCT,584012647694946,1,P_NzI3NDUxNDRfMTM4NDM1MDQ5NC4yMTUx
302,Card Campaign,2014-01-12 22:05:11,-10000,CHOPPIES GABANE ROAD BOT BW,DEDUCT,584012651116258,1,P_NzQzNTQzODFfMTM4NzM2Njg5NS4xODAy


In [29]:
# We start the iteration through the dataframes comparing each index within both dataframes. This is possible because we have sorted the data
# And We dropped duplicates too
index = 0
clientMatch = 0
clientMismatch = 0
tutukaMatch = 0
tutukaMismatch = 0
maxIndex = getMaxSize()

while(index < maxIndex):
    #print(f'index:{index}')
    clientRow = sortedClient.iloc[index]
    tutukaRow = sortedTutuka.iloc[index]
    #print(f'{clientRow["TransactionID"]}    {tutukaRow["TransactionID"]}')
    #First only check if the TransactionsID are the same. If both match, continue to check all the other data.
    if(clientRow["TransactionID"] == tutukaRow["TransactionID"]):
        #print("IDs iguales")
        if(compareRowData(clientRow, tutukaRow)):
            #print("Match perfecto")
            #Add one match to each file because the transaction is on both. So is a match for both of them
            clientMatch += 1
            tutukaMatch += 1
        else:
            #If they have the same ID but different information it may be possible that script is has switched the transactionDescription in one file.
            #But the transaction may still have a match
            #So we check with the next row if we find the same description.

            #print(f'Posible inversion de descripcion en {index}')
            #print(f'{clientRow["TransactionID"]}    {tutukaRow["TransactionID"]}')
            tutukaNextRow = sortedTutuka.iloc[index + 1]
            clientNextRow = sortedClient.iloc[index + 1]
            if(compareRowData(clientRow, tutukaNextRow)):
                #print("Si hay matches invertidos")
                #Add new matches
                clientMatch += 1
                tutukaMatch += 1
                if(compareRowData(clientNextRow, tutukaRow)):
                    clientMatch += 1
                    tutukaMatch += 1
                else:
                    clientMismatch += 1
                    tutukaMismatch += 1
                #Add one to index because we already checked the next row
                index += 1
            else:
                print(f"No es match. Posible {index} {clientRow}")
                clientMismatch += 1
                tutukaMismatch += 1
            #print("---------------------------------")               
    else:
        #print(f'{clientRow["TransactionID"]}    {tutukaRow["TransactionID"]}')
        if(clientRow["TransactionID"] > tutukaRow["TransactionID"]):
            #print(f'Mayor   index:{index}')
            sortedClient = insertEmptyRow(sortedClient, index + 1)
            maxIndex = getMaxSize()
            tutukaMismatch += 1
            
        else:
            #print("Menor")
            maxIndex = getMaxSize()
            sortedTutuka = insertEmptyRow(sortedTutuka, index + 1)
            clientMismatch += 1

    index += 1

print(f'Client file matches: {clientMatch}')
print(f'Client file Mismatches: {clientMismatch}')
print(f'Tutuka file matches: {tutukaMatch}')
print(f'Tutuka file Mismatches: {tutukaMismatch}')

304
304
Max:304

No es match. Posible 46 ProfileName                                            Card Campaign
TransactionDate                                  2014-01-12 15:03:05
TransactionAmount                                              -6984
TransactionNarrative      128552 P G TIMBERS MAUN   BOTSWANA      BW
TransactionDescription                                        DEDUCT
TransactionID                                         84012397854064
TransactionType                                                    0
WalletReference                                                  NaN
Name: 46, dtype: object
No es match. Posible 62 ProfileName                                    Card Campaign
TransactionDate                          2014-01-12 10:39:59
TransactionAmount                                     -11490
TransactionNarrative                    *Edgars SDTN      ZA
TransactionDescription                                DEDUCT
TransactionID                                164012239

IndexError: single positional indexer is out-of-bounds

In [30]:
sortedClient.head(92)

,ProfileName,TransactionDate,TransactionAmount,TransactionNarrative,TransactionDescription,TransactionID,TransactionType,WalletReference
0,Card Campaign,2014-01-12 08:24:56,-10250,Sunset Butchery 100343 Francistown BW,DEDUCT,4012304765521,0,P_NzI4Njk3MTBfMTM4MTQ5MTEwNy4wNTMy
1,Card Campaign,2014-01-12 08:41:20,-3475,Choppies Superst102145 Lobatse BW,DEDUCT,4012314602553,0,P_NzU3OTY0MzBfMTM4NzIwMzAxOS4zMzYz
2,Card Campaign,2014-01-12 12:56:25,4310,766831 SHOPRITE GABORONE BOTSWANA BW,REVERSAL,4012321856252,0,P_NzI0MTE0MjJfMTM4ODEzMTA0Mi42MTI3
3,Card Campaign,2014-01-12 12:56:25,-4310,766831 SHOPRITE GABORONE BOTSWANA BW,DEDUCT,4012321856252,0,P_NzI0MTE0MjJfMTM4ODEzMTA0Mi42MTI3
4,Card Campaign,2014-01-12 12:57:31,-4310,766831 SHOPRITE GABORONE BOTSWANA BW,DEDUCT,4012322510401,0,P_NzI0MTE0MjJfMTM4ODEzMTA0Mi42MTI3
...,...,...,...,...,...,...,...,...
87,Card Campaign,2014-01-12 22:08:07,-2695,921568 MOCHUDI SPAR BOTSWANA BW,DEDUCT,164012652889581,0,P_NzQ4NzY3NzBfMTM4OTI2MzIxNi45NTk2
88,Card Campaign,2014-01-12 08:25:03,-5000,616432 WHITE CITY FILLING BOTSWANA BW,DEDUCT,284012159042546,0,P_NzQzMDA2NjlfMTM4OTQzODQ2NS43OTUz
89,Card Campaign,2014-01-12 10:43:22,-2000,563667 ENGEN TRUCK STOP BOTSWANA BW,DEDUCT,284012242033591,0,P_NzUzNTQ1NDNfMTM4MzgxNDM1Ni45NDI0
90,Card Campaign,2014-01-12 12:06:59,-7000,340579 BOTSHELO SERVICE S BOTSWANA BW,DEDUCT,284012292196971,0,P_NzI5NTc5MzdfMTM3Nzc2MjY2Ny4zODU=
